# 一些实例

本文主要记录一些平常自己写的，在实际中使用的代码。

## 计算流域平均气象时间序列数据

这里以Daymet 2天的网格数据，CAMELS 多个流域为例，计算这些流域这两天每日的forcing数据流域平均值。

因为CAMELS文件比较大，所以这里没有传到github上，需要手动从[这里](https://ral.ucar.edu/sites/default/files/public/product-tool/camels-catchment-attributes-and-meteorology-for-large-sample-studies-dataset-downloads/basin_set_full_res.zip)下载 CAMELS 的shpfile ，然后在本文件夹下创建一个large_files文件夹，并将下好的CAMELS shpfile放到其中。

也可以选择上传到GEE asset上，然后直接调用。

In [1]:
import ee
import geemap
Map = geemap.Map(center=[40, -100], zoom=4)
Map

Enter verification code:  4/1AX4XfWgjIE6EFcTTAZrRV4fRqQWS0hrKKJknOm0LiO1nlXcWQDN7iGjXXLE



Successfully saved authorization token.


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [2]:
# Add Earth Engine dataset
daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4")

In [3]:
# 本地文件
# camels_shp = 'large_files/HCDN_nhru_final_671.shp'
# camels = geemap.shp_to_ee(camels_shp)
# 远程asset上
camels = ee.FeatureCollection("users/wenyu_ouyang/HCDN_nhru_final_671")
camels

In [4]:
#maybe better to use Number to replace js number
year = ee.Number(2000)
month = ee.Number(1)
day = ee.Number(1)
start_date = ee.Date.fromYMD(year, month, day)
end_date = start_date.advance(2, 'day')
end_date

In [5]:
days_num = end_date.difference(start_date, 'day')
# count day from zero, and ee.List.sequence is a closed interval
days = ee.List.sequence(ee.Number(0),days_num.add(-1))
# get Imagecollection and filter, choose two days for test
daymet_days = daymet.filter(ee.Filter.date(start_date, end_date))

# show maximumTemperature, just for test
maximumTemperature = daymet_days.select('tmax')
maximumTemperatureVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}
Map.setCenter(-110.21, 35.1, 4)
Map.addLayer(maximumTemperature, maximumTemperatureVis, 'Maximum Temperature')

In [8]:
def nestedMappedReducer(featCol, imgCol):
    def mapReducerOverImgCol(feat):
        def imgReducer(img):
            vals = img.reduceRegion(
                reducer = ee.Reducer.mean(),
                geometry = feat.geometry(),
                scale = 1000
            )
            return ee.Feature(None, vals).set({
                "system:time_start": img.get("system:time_start"),
                "hru_id" : feat.get("hru_id")
            })
        return imgCol.map(imgReducer);
    return featCol.map(mapReducerOverImgCol).flatten()

执行函数并导出到 google drive，这样本地可以关闭，远程也在运行了，很适合较长时间的计算。

In [9]:
daymet_regions = nestedMappedReducer(camels, daymet_days)
#export to google drive
geemap.ee_export_vector_to_drive(
    ee_object=daymet_regions, description="daymet_camels_mean_20000101-02new", folder="export", file_format="csv", 
    selectors=["hru_id","system:time_start","dayl","prcp","srad","swe","tmax","tmin","vp"]
)

Exporting daymet_camels_mean_20000101-02new...


来自GEE的建议，使用reduceRegions可能比reduceRegion更快一些，但是这里不方便设置每个feature的time_start和hru_id，应该还是需要对featurecollection加一个map函数，所以暂时就不尝试了。

```Javascript
results = daymet_days.map(function(img) {
    return img.reduceRegions({
        ...
    });
});
```

上面给出的是JS代码，关于map和reduce，尤其是嵌套的js代码转python代码，可以参考这里：https://gis.stackexchange.com/questions/365121/how-to-nest-mapped-functions-with-the-earth-engine-python-api

## 小时尺度气象数据平均到日尺度

这里以NLDAS数据为例，将shpfile中的小时尺度数据均化到日尺度。shpfile可以仍选择CAMELS，这里使用了自己生成的，生成过程请参考 AutoGIS/9.1-gallery-vector.ipynb 中例五，这里直接使用结果得到的shpfile了，我已经将其上传至自己的GEE上了，所以就不使用本地的了。

In [27]:
import ee
import geemap
Map = geemap.Map(center=[40, -100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [2]:
# Add Earth Engine dataset
nldas = ee.ImageCollection("NASA/NLDAS/FORA0125_H002")

In [3]:
# 远程asset上
shpfiles = ee.FeatureCollection("users/wenyu_ouyang/site_nobs_DO")
shpfiles

In [4]:
year = ee.Number(2000)
month = ee.Number(1)
day = ee.Number(1)
start_date = ee.Date.fromYMD(year, month, day)
end_date = start_date.advance(2, 'day')
end_date

In [8]:
days_num = end_date.difference(start_date, 'day')
# count day from zero, and ee.List.sequence is a closed interval
days = ee.List.sequence(ee.Number(0),days_num.add(-1))
# get Imagecollection and filter, choose two days for test
nldas_2d = nldas.filter(ee.Filter.date(start_date, end_date))

# show temperature, just for test
temperature = nldas_2d.select('temperature');
temperatureVis = {
  'min': -5.0,
  'max': 40.0,
  'palette': ['3d2bd8', '4e86da', '62c7d8', '91ed90', 'e4f178', 'ed6a4c'],
};
Map.addLayer(temperature, temperatureVis, 'Temperature')
# 查看上面的地图

In [11]:
# forcing variables that will be calculated for its avg
avg_forcings=nldas_2d.select('temperature','specific_humidity','pressure','wind_u','wind_v','longwave_radiation','shortwave_radiation')
avg_forcings.limit(2)

In [12]:
# forcing variables that will be calculated for its sum
sum_forcings=nldas_2d.select('potential_energy','potential_evaporation','total_precipitation')
sum_forcings.limit(2)

In [22]:
def daysMapImgsAvgReduce(dayCol, imgCol, start_day):
    def dayAvgReducerOverImgCol(oneDay):
        start = start_day.advance(ee.Number(oneDay), 'day');
        end = start_day.advance(ee.Number(oneDay).add(ee.Number(1)), 'day');
        return imgCol.filter(ee.Filter.date(start,end)).reduce(ee.Reducer.mean()).set({
            'day_of_all_years': oneDay
        })
    return dayCol.map(dayAvgReducerOverImgCol)

In [24]:
avg_days = ee.ImageCollection(daysMapImgsAvgReduce(days, avg_forcings, start_date))

In [23]:
def daysMapImgsSumReduce(dayCol, imgCol, start_day):
    def daySumReducerOverImgCol(oneDay):
        start = start_day.advance(ee.Number(oneDay), 'day');
        end = start_day.advance(ee.Number(oneDay).add(ee.Number(1)), 'day');
        return imgCol.filter(ee.Filter.date(start,end)).reduce(ee.Reducer.sum()).set({
            'day_of_all_years': day
        })
    return dayCol.map(daySumReducerOverImgCol)

In [25]:
sum_days =  ee.ImageCollection(daysMapImgsSumReduce(days, sum_forcings, start_date))

In [28]:
# show avg temperature of all days, just for test
tmpr_avg = avg_days.select('temperature_mean').reduce(ee.Reducer.mean())
temperature_avg = tmpr_avg.select('temperature_mean_mean')
Map.addLayer(temperature_avg, temperatureVis, 'Temperature_2d_avg')

最后求一下流域平均：

In [29]:
def nestedMappedReducerNldas(featCol, imgCol, scaleNum):
    def mapReducerOverImgColNldas(feat):
        def imgReducerNldas(img):
            vals = img.reduceRegion(
                reducer = ee.Reducer.mean(),
                geometry = feat.geometry(),
                scale = scaleNum
            )
            return ee.Feature(None, vals).set({
                "time_start": img.get("day_of_all_years"),
                "gage_id" : feat.get("GAGE_ID")
            })
        return imgCol.map(imgReducerNldas);
    return featCol.map(mapReducerOverImgColNldas).flatten()

In [30]:
# 0.125 degree approximately equals to 13875m
avg_day_regions = nestedMappedReducerNldas(shpfiles, avg_days, 13875)
#export to google drive
geemap.ee_export_vector_to_drive(
    ee_object=avg_day_regions, description="nldas_do_mean_20000101-02", folder="export", file_format="csv", 
    selectors=["gage_id","time_start","temperature_mean","specific_humidity_mean","pressure_mean","wind_u_mean","wind_v_mean","longwave_radiation_mean","shortwave_radiation_mean"]
)

Exporting nldas_do_mean_20000101-02...


In [31]:
avg_day_regions_4sum = nestedMappedReducerNldas(shpfiles, sum_days, 13875)
#export to google drive
geemap.ee_export_vector_to_drive(
    ee_object=avg_day_regions_4sum, description="nldas_do4sum_mean_20000101-02", folder="export", file_format="csv", 
    selectors=["gage_id","time_start","potential_energy_sum","potential_evaporation_sum","total_precipitation_sum"]
)

Exporting nldas_do4sum_mean_20000101-02...


下面是一个整合的一年计算的实例供实际使用：

In [36]:
import ee
import geemap
year_num = 1980
year = ee.Number(year_num)
month = ee.Number(1)
day = ee.Number(1)
start_date = ee.Date.fromYMD(year, month, day)
end_date = start_date.advance(1, 'year')
days_num = end_date.difference(start_date, 'day')
days = ee.List.sequence(ee.Number(0),days_num.add(-1))

nldas = ee.ImageCollection("NASA/NLDAS/FORA0125_H002")
shpfiles = ee.FeatureCollection("users/wenyu_ouyang/site_nobs_DO")

nldas_days = nldas.filter(ee.Filter.date(start_date, end_date))
avg_forcings=nldas_days.select('temperature','specific_humidity','pressure','wind_u','wind_v','longwave_radiation','convective_fraction','shortwave_radiation')
sum_forcings=nldas_days.select('potential_energy','potential_evaporation','total_precipitation')
avg_days = ee.ImageCollection(daysMapImgsAvgReduce(days, avg_forcings, start_date))
sum_days =  ee.ImageCollection(daysMapImgsSumReduce(days, sum_forcings, start_date))
# 0.125 degree approximately equals to 13875m
avg_day_regions = nestedMappedReducerNldas(shpfiles, avg_days, 13875)
#export to google drive
geemap.ee_export_vector_to_drive(
    ee_object=avg_day_regions, description="nldas_do_avg_mean_" + str(year_num), folder="NLDAS", file_format="csv", 
    selectors=["gage_id","time_start","temperature_mean","specific_humidity_mean","pressure_mean","wind_u_mean","wind_v_mean","longwave_radiation_mean","convective_fraction_mean","shortwave_radiation_mean"]
)
avg_day_regions_4sum = nestedMappedReducerNldas(shpfiles, sum_days, 13875)
#export to google drive
geemap.ee_export_vector_to_drive(
    ee_object=avg_day_regions_4sum, description="nldas_do_sum_mean_" + str(year_num), folder="NLDAS", file_format="csv", 
    selectors=["gage_id","time_start","potential_energy_sum","potential_evaporation_sum","total_precipitation_sum"]
)

Exporting nldas_do_avg_mean_1980...
Exporting nldas_do_sum_mean_1980...


上面这个完整实例在GEE中运行时间很长，如果不想让其运行，在GEE中手动终止任务即可。